# Hyper-parameters

In [16]:
dir_name = "Model1_2D" #location to save the model in Google Drive

In [17]:
n_Gen = 6 #number of generators
h_Dim = 128 #dimention of hidden layers
latent_dim =  64 # dimention of input noise
size_dataset =  200000 #size of dataset
batch_size = 128 #number of batches

steps_per_epoch = (size_dataset//batch_size)//n_Gen

# Adding Libraries

In [18]:
import matplotlib.pyplot as plt
from sklearn import mixture
import math

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Model
from google.colab import output

import os

In [19]:
# for saving GIF
!pip install pygifsicle
!sudo apt-get install gifsicle
import imageio
import glob
from pygifsicle import optimize
output.clear()
print("Import Done")

Import Done


# To see if we have a GPU

In [20]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

if tf.test.gpu_device_name() == '/device:GPU:0':
  print("Using the GPU")
else:
  print("Using the CPU")

Num GPUs Available:  0
Using the CPU


# Mount Google drive to save model and data

In [21]:
from google.colab import drive
drive.mount('/content/drive')

if os.path.exists(f'/content/drive/MyDrive/{dir_name}') == False:
    os.mkdir(f'/content/drive/MyDrive/{dir_name}')
    os.mkdir(f'/content/drive/MyDrive/{dir_name}/Charts')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2D Gaussian Dataset

In [22]:
def dataset2d_func(size_dataset = 200000, num_mixtures = 6, radius = 5, std = 0.1, random_state = None):
    thetas = tf.linspace(0., 2 * math.pi, num_mixtures + 1)[:num_mixtures]
    centers = tf.stack([radius * tf.cos(thetas), radius * tf.sin(thetas)], axis = -1)

    gmm = mixture.GaussianMixture(n_components = num_mixtures, covariance_type = 'diag')
    gmm.means_ = centers
    gmm.covariances_ = tf.constant([[std, std]] * num_mixtures)**2
    gmm.weights_ = tf.constant([1/num_mixtures] * num_mixtures)
    X = gmm.sample(size_dataset)
    return X[0]

# Some Functions

##### A function that produces uniform random noise in the range of [-1,1] of size [n_gen, batch_size, latent_dim] as the generators' input

In [23]:
from tensorflow_probability import distributions as tfd

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, batch_size, n_Gen):
    # Multivariate normal diagonal distribution
    mvn = tfd.MultivariateNormalDiag(
        loc=[0]*latent_dim,
        scale_diag=[1.0]*latent_dim)
    
    noise = []
    for i in range(n_Gen):
        # Some samples from MVN
        x_input = mvn.sample(batch_size)
        noise.append(x_input)
    return noise

##### A callback which runs at end of each epoch to save and plot the results

In [24]:
class GANMonitor1(tf.keras.callbacks.Callback):
    def __init__(self, dataset, plot_freq = 1, num_samples = 200000, latent_dim = 64, n_Gen = 6, dir_name = 'Model'):
        self.dataset = dataset
        self.num_samples = num_samples
        self.latent_dim = latent_dim
        self.n_Gen = n_Gen
        self.dir_name = dir_name
        self.plot_freq = plot_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.plot_freq == 0:
            bin = 500
            random_latent_vectors = generate_latent_points(self.latent_dim, self.num_samples, self.n_Gen)

            generated_samples = []
            for g in range(self.n_Gen):
                generated_samples.append(self.model.generators[g](random_latent_vectors[g]))

            combined_generated_samples = tf.concat([generated_samples[g] for g in range(self.n_Gen)], axis=0)

            _, ax = plt.subplots(nrows = 2, ncols = 4, figsize = ([12.8, 7.2]), sharey = True)

            ax[0, 0].hist2d(self.dataset[:,0],
                            self.dataset[:,1], 
                            bins=(bin, bin),
                            cmap=plt.cm.YlOrBr,
                            range = [[-150, 150], [-150, 150]],
                            density = True)
            ax[0, 0].set_aspect('equal')
            ax[0, 0].set_title('Dataset')

            ax[0, 1].hist2d(generated_samples[0].numpy()[:,0],
                            generated_samples[0].numpy()[:,1],
                            bins=(bin, bin),
                            cmap=plt.cm.YlOrBr,
                            range = [[-150, 150], [-150, 150]],
                            density = True)
            ax[0, 1].set_aspect('equal')
            ax[0, 1].set_title('Generator 1')

            ax[0, 2].hist2d(generated_samples[1].numpy()[:,0],
                            generated_samples[1].numpy()[:,1], 
                            bins=(bin, bin),
                            cmap=plt.cm.YlOrBr,
                            range = [[-150, 150], [-150, 150]], 
                            density = True)
            ax[0, 2].set_aspect('equal')
            ax[0, 2].set_title('Generator 2')

            ax[0, 3].hist2d(generated_samples[2].numpy()[:,0], 
                            generated_samples[2].numpy()[:,1], 
                            bins=(bin, bin), 
                            cmap=plt.cm.YlOrBr, 
                            range = [[-150, 150], [-150, 150]],
                            density = True)
            ax[0, 3].set_aspect('equal')
            ax[0, 3].set_title('Generator 3')

            ax[1, 1].hist2d(generated_samples[3].numpy()[:,0],
                            generated_samples[3].numpy()[:,1],
                            bins=(bin, bin),
                            cmap=plt.cm.YlOrBr, 
                            range = [[-150, 150], [-150, 150]],
                            density = True)
            ax[1, 1].set_aspect('equal')
            ax[1, 1].set_title('Generator 4')

            ax[1, 2].hist2d(generated_samples[4].numpy()[:,0],
                            generated_samples[4].numpy()[:,1],
                            bins=(bin, bin), 
                            cmap=plt.cm.YlOrBr,
                            range = [[-150, 150], [-150, 150]],
                            density = True)
            ax[1, 2].set_aspect('equal')
            ax[1, 2].set_title('Generator 5')

            ax[1, 3].hist2d(generated_samples[5].numpy()[:,0],
                            generated_samples[5].numpy()[:,1],
                            bins=(bin, bin), 
                            cmap=plt.cm.YlOrBr,
                            range = [[-150, 150], [-150, 150]],
                            density = True)
            ax[1, 3].set_aspect('equal')
            ax[1, 3].set_title('Generator 6')

            ax[1, 0].hist2d(combined_generated_samples.numpy()[:,0],
                            combined_generated_samples.numpy()[:,1], 
                            bins=(bin, bin), 
                            cmap=plt.cm.YlOrBr, 
                            range = [[-150, 150], [-150, 150]],
                            density = True)
            ax[1, 0].set_aspect('equal')
            ax[1, 0].set_title('All Generators')

            plt.subplots_adjust(hspace = 0.05, wspace = 0.1)
            plt.savefig(f'/content/drive/MyDrive/{self.dir_name}/Charts/chart_{(epoch + 1):04}.png', dpi=200, format="png")
            
            # To show the plots in colab comment line below and uncomment the next line
            # plt.close()
            plt.show()

##### Loss function for the generators based on the MAD_GAN paper

In [25]:
def Generators_loss_function(y_true, y_pred):
    logarithm = -tf.math.log(y_pred[:,-1] + 1e-15)
    return tf.reduce_mean(logarithm, axis=-1)

# Defining Discriminator Model

In [26]:
# define the standalone discriminator model
def define_discriminator(n_Gen, h_Dim):
    inp = Input(shape = (2,))
    x = Dense(h_Dim, input_shape=(2,), activation = 'relu')(inp)
    out = Dense(n_Gen+1, activation="softmax")(x)
    model = Model(inp, out, name="discriminator")
    return model

# Defining Generators Model

In [27]:
def define_generators(n_Gen, latent_dim, h_Dim):
    mid_layer1 = Dense(h_Dim, name = "hidden_1", activation = 'relu', input_shape=(latent_dim,))

    models = []
    for g in range(n_Gen):
        input = Input(shape=(latent_dim,), dtype = tf.float32, name=f"input_{g}")
        x = mid_layer1(input)
        x = BatchNormalization()(x)

        x = Dense(h_Dim, activation = 'relu')(x)
        x = BatchNormalization()(x)

        x = Dense(2, dtype = tf.float32, name=f"generator_output_{g}")(x)

        models.append(Model(input, x, name = f"generator{g}"))
    return models

# Defining MADGAN Class for training via keras

In [28]:
class MADGAN(tf.keras.Model):
    def __init__(self, discriminator, generators, latent_dim, n_Gen):
        super(MADGAN, self).__init__()
        self.discriminator = discriminator
        self.generators = generators
        self.latent_dim = latent_dim
        self.n_Gen = n_Gen

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(MADGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    def train_step(self, data):
        X = data
        # Get the batch size
        batch_size = tf.shape(X)[0]
        # Sample random points in the latent space
        random_latent_vectors = generate_latent_points(self.latent_dim, batch_size//self.n_Gen, self.n_Gen)
        # Decode them to fake generator output
        x_generator = []
        for g in range(self.n_Gen):
            x_generator.append(self.generators[g](random_latent_vectors[g]))

        # Combine them with real samples
        combined_samples = tf.concat([x_generator[g] for g in range(self.n_Gen)] + 
                                     [X], 
                                     axis=0
                                     )

        # Assemble labels discriminating real from fake samples
        labels = tf.concat([tf.one_hot(g * tf.ones(batch_size//self.n_Gen, dtype=tf.int32), self.n_Gen + 1) for g in range(self.n_Gen)] + 
                    [tf.one_hot(self.n_Gen * tf.ones(batch_size, dtype=tf.int32), self.n_Gen + 1)], 
                    axis=0
                    )

        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(shape = tf.shape(labels), minval = -1, maxval = 1)

        #######################
        # Train Discriminator #
        #######################
        
        # make weights in the discriminator trainable
        with tf.GradientTape() as tape:
            # Discriminator forward pass
            predictions = self.discriminator(combined_samples)

            # Compute the loss value
            # (the loss function is configured in `compile()`)
            d_loss = self.d_loss_fn(labels, predictions)

        # Compute gradients
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)

        # Update weights
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        #######################
        #   Train Generator   #
        #######################

        # Assemble labels that say "all real samples"
        misleading_labels =  tf.one_hot(self.n_Gen * tf.ones(batch_size//self.n_Gen, dtype=tf.int32), self.n_Gen + 1)

        # (note that we should *not* update the weights of the discriminator)!
        g_loss_list = []
        for g in range(self.n_Gen):
            with tf.GradientTape() as tape:
                # Generator[g] and discriminator forward pass
                predictions = self.discriminator(self.generators[g](random_latent_vectors[g]))

                # Compute the loss value
                # (the loss function is configured in `compile()`)
                g_loss = self.g_loss_fn(misleading_labels, predictions)

            # Compute gradients
            grads = tape.gradient(g_loss, self.generators[g].trainable_weights)
            # Update weights
            self.g_optimizer[g].apply_gradients(zip(grads, self.generators[g].trainable_weights))
            g_loss_list.append(g_loss)

        mydict = {f"g_loss{g}": g_loss_list[g] for g in range(self.n_Gen)}
        mydict.update({"d_loss": d_loss})
        return mydict

# Creating Model and training it

In [29]:
# Loading data
data = dataset2d_func(size_dataset, num_mixtures = 6, radius = 100, std = 10)
# Changing numpy dataset to tf.DATASET type and Shuffling dataset for training
dataset = tf.data.Dataset.from_tensor_slices(data) 
dataset = dataset.repeat().shuffle(10 * size_dataset, reshuffle_each_iteration=True).batch(n_Gen * batch_size, drop_remainder=True)

# Creating Discriminator and Generator
discriminator = define_discriminator(n_Gen, h_Dim)
generators = define_generators(n_Gen, latent_dim, h_Dim)

# creating MADGAN
madgan = MADGAN(discriminator = discriminator, generators = generators, 
                latent_dim = latent_dim, n_Gen = n_Gen)

madgan.compile(
    d_optimizer = Adam(learning_rate=2e-4, beta_1=0.5),
    g_optimizer = [Adam(learning_rate=2e-4, beta_1=0.5) for g in range(n_Gen)],
    d_loss_fn = CategoricalCrossentropy(),
    g_loss_fn = Generators_loss_function
)

# saved model directory
checkpoint_filepath = f'/content/drive/MyDrive/{dir_name}/checkpoint'

# callbacks are functions that run at end of each epoch
my_callbacks = [
    # This callback is for ploting generators' output every epoch
    GANMonitor1(dataset = data, plot_freq = 2, num_samples = size_dataset//n_Gen, latent_dim = latent_dim, n_Gen = n_Gen, dir_name = dir_name),
    # This callback is for Saving the model every 15 epochs
    tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_filepath, save_freq = 20, save_weights_only=True),
]

# # Loading previous saved model for resume training
# if os.path.exists(checkpoint_filepath):
#     madgan.load_weights(checkpoint_filepath)

# train the model
madgan.fit(dataset, epochs = 500, initial_epoch = 0, steps_per_epoch = steps_per_epoch, verbose = 1, callbacks = my_callbacks)

Epoch 1/500


KeyboardInterrupt: ignored

# Saving GIF file

In [ ]:
anim_file = 'madgan.gif'

with imageio.get_writer(f'/content/drive/MyDrive/{dir_name}/{anim_file}', mode='I') as writer:
    filenames = glob.glob(f'/content/drive/MyDrive/{dir_name}/Charts/chart*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        # image = image[::4,::4,:]
        writer.append_data(image)
        writer.append_data(image)
    for i in range(20):
        writer.append_data(image)   

# Reduce GIF size
optimize(f'/content/drive/MyDrive/{dir_name}/{anim_file}')